# Traffic Incident Management in Nashville, TN

### Loading the libraries

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from datetime import datetime
import math
import sys
import re

from shapely.geometry import Point
from shapely.geometry import LineString
import geopandas as gpd
import descartes
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from folium import plugins
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime

%matplotlib inline

### Importing the data

In [2]:
traffic_df = pd.read_csv('../data/MASTER File 2021.csv')
d_weather = pd.read_csv('../data/Daily Weather Formatted.csv')
h_weather = pd.read_csv('../data/Hourly Weather Formatted.csv')

C:\Users\suzan\anaconda3\envs\geospatial\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### Reviewing the Traffic Dataframe

In [3]:
print(traffic_df.shape)

(33730, 27)


In [4]:
traffic_df.head(2)

,Region,Incident ID,Incident Type,Full Location,Start Date,Start Time,Duration ICT,Lanes Blocked,Blockage Duration RCT,Response Time,...,Description,Inc MM,Latitude,Longitude,Severity Lanes,Severity Duration,Severity,Temp,Prec,Condition
0,3,726,MultiVehicle Crash,"DAVIDSON on Interstate 40 Westbound, Beyond MI...",11/12/2020,10:25:06 AM,100,0,0,0,...,MILE MARKER 213.2,213.2,36.140377,-86.723936,Level 1,Level 1,Level 1,55.0,0.0,Clear
1,3,728,Disabled Vehicle,"DAVIDSON on Interstate 65 Northbound, Before M...",11/12/2020,10:35:05 AM,102.3666667,0,0,0.066666667,...,MILE MARKER 80.6,80.6,36.122863,-86.773113,Level 1,Level 1,Level 1,55.0,0.0,Clear


In [5]:
traffic_df['Start Date'].value_counts()

7/9/2021      146
4/7/2021      145
3/2/2021      141
7/29/2021     140
2/22/2021     140
             ... 
3/14/2021      51
9/12/2021      50
5/29/2021      46
12/27/2020     41
1/24/2021      31
Name: Start Date, Length: 365, dtype: int64

### Converting the datetime data type from object to datetime64 in traffic dataframe

In [6]:
traffic_df['Start Date'] = pd.to_datetime(traffic_df['Start Date'])

In [7]:
traffic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33730 entries, 0 to 33729
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Region                 33730 non-null  int64         
 1   Incident ID            33730 non-null  int64         
 2   Incident Type          33730 non-null  object        
 3   Full Location          33730 non-null  object        
 4   Start Date             33730 non-null  datetime64[ns]
 5   Start Time             33730 non-null  object        
 6   Duration ICT           33730 non-null  object        
 7   Lanes Blocked          33730 non-null  int64         
 8   Blockage Duration RCT  33730 non-null  object        
 9   Response Time          33730 non-null  object        
 10  Rural/
Urban           33730 non-null  object        
 11  Key                    33730 non-null  int64         
 12  County                 33730 non-null  object        
 13  I

In [8]:
d_weather.head(1)

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,sealevelpressure,cloudcover,visibility,sunrise,sunset,moonphase,conditions,description,icon,stations
0,nashville tennessee,2020-11-01,61.6,41.6,54.5,61.6,36.6,53.5,34.1,49.95,...,1025.2,22.4,9.9,2020-11-01T06:10:26,2020-11-01T16:50:26,0.51,Clear,Partly cloudy throughout the day.,Clear,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"


### Converting the datetime data type from object to datetime64 in daily weather dataframe 

In [9]:
d_weather['datetime'] = pd.to_datetime(d_weather['datetime'])

### Merging traffic and weather dataframe

In [10]:
traffic_df = pd.merge(traffic_df, d_weather, left_on = 'Start Date', right_on = 'datetime', how = 'inner')

In [11]:
traffic_df.head()

,Region,Incident ID,Incident Type,Full Location,Start Date,Start Time,Duration ICT,Lanes Blocked,Blockage Duration RCT,Response Time,...,sealevelpressure,cloudcover,visibility,sunrise,sunset,moonphase,conditions,description,icon,stations
0,3,726,MultiVehicle Crash,"DAVIDSON on Interstate 40 Westbound, Beyond MI...",2020-11-12,10:25:06 AM,100,0,0,0,...,1017.7,5.1,9.9,2020-11-12T06:21:29,2020-11-12T16:40:48,0.95,Clear,Clear conditions throughout the day.,Clear,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
1,3,728,Disabled Vehicle,"DAVIDSON on Interstate 65 Northbound, Before M...",2020-11-12,10:35:05 AM,102.3666667,0,0,0.066666667,...,1017.7,5.1,9.9,2020-11-12T06:21:29,2020-11-12T16:40:48,0.95,Clear,Clear conditions throughout the day.,Clear,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
2,3,732,Abandoned Vehicle,"DAVIDSON on Interstate 24 Eastbound, Before MI...",2020-11-12,10:59:43 AM,"1,690",0,0,0,...,1017.7,5.1,9.9,2020-11-12T06:21:29,2020-11-12T16:40:48,0.95,Clear,Clear conditions throughout the day.,Clear,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
3,3,735,Disabled Vehicle,"RUTHERFORD on Interstate 24 Eastbound, Before ...",2020-11-12,11:04:04 AM,55,0,0,0,...,1017.7,5.1,9.9,2020-11-12T06:21:29,2020-11-12T16:40:48,0.95,Clear,Clear conditions throughout the day.,Clear,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
4,3,736,Abandoned Vehicle,"DAVIDSON on Interstate 40 Westbound, Before MI...",2020-11-12,11:05:09 AM,"1,224",1,12,0,...,1017.7,5.1,9.9,2020-11-12T06:21:29,2020-11-12T16:40:48,0.95,Clear,Clear conditions throughout the day.,Clear,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"


In [12]:
rain_df = traffic_df.loc[traffic_df.conditions =='Rain']

In [13]:
rain_df.head(2)

,Region,Incident ID,Incident Type,Full Location,Start Date,Start Time,Duration ICT,Lanes Blocked,Blockage Duration RCT,Response Time,...,sealevelpressure,cloudcover,visibility,sunrise,sunset,moonphase,conditions,description,icon,stations
240,3,1002,MultiVehicle Crash,"WILLIAMSON on Interstate 65 Southbound, Beyond...",2020-11-15,2:56:20 AM,29.26666667,0,0,0,...,1016.0,43.6,9.7,2020-11-15T06:24:32,2020-11-15T16:38:44,0.0,Rain,Clearing in the afternoon with rain clearing l...,Rain,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
241,3,1003,Disabled Vehicle,"DAVIDSON on Interstate 65 Northbound, Before M...",2020-11-15,3:32:54 AM,83.18333333,0,0,0,...,1016.0,43.6,9.7,2020-11-15T06:24:32,2020-11-15T16:38:44,0.0,Rain,Clearing in the afternoon with rain clearing l...,Rain,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"


In [14]:
snow_df = traffic_df.loc[traffic_df.conditions == 'Snow']

In [15]:
snow_df.head(2)

,Region,Incident ID,Incident Type,Full Location,Start Date,Start Time,Duration ICT,Lanes Blocked,Blockage Duration RCT,Response Time,...,sealevelpressure,cloudcover,visibility,sunrise,sunset,moonphase,conditions,description,icon,stations
4843,3,7006,Disabled Vehicle,"DAVIDSON on Interstate 24 Eastbound, Before MI...",2021-01-11,12:32:28 AM,"2,542",0,0,418,...,1027.6,94.7,7.5,2021-01-11T06:58:19,2021-01-11T16:52:24,0.99,Snow,Cloudy skies throughout the day with late afte...,Snow,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
4844,3,7007,Disabled Vehicle,"DAVIDSON on Interstate 65 Northbound, Before M...",2021-01-11,12:36:08 AM,39.36666667,0,0,8.8,...,1027.6,94.7,7.5,2021-01-11T06:58:19,2021-01-11T16:52:24,0.99,Snow,Cloudy skies throughout the day with late afte...,Snow,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"


In [16]:
traffic_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33730 entries, 0 to 33729
Data columns (total 54 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Region                 33730 non-null  int64         
 1   Incident ID            33730 non-null  int64         
 2   Incident Type          33730 non-null  object        
 3   Full Location          33730 non-null  object        
 4   Start Date             33730 non-null  datetime64[ns]
 5   Start Time             33730 non-null  object        
 6   Duration ICT           33730 non-null  object        
 7   Lanes Blocked          33730 non-null  int64         
 8   Blockage Duration RCT  33730 non-null  object        
 9   Response Time          33730 non-null  object        
 10  Rural/
Urban           33730 non-null  object        
 11  Key                    33730 non-null  int64         
 12  County                 33730 non-null  object        
 13  I

In [17]:
clear_df = traffic_df.loc[traffic_df.conditions == 'Clear']

In [18]:
clear_df.head(2)

,Region,Incident ID,Incident Type,Full Location,Start Date,Start Time,Duration ICT,Lanes Blocked,Blockage Duration RCT,Response Time,...,sealevelpressure,cloudcover,visibility,sunrise,sunset,moonphase,conditions,description,icon,stations
0,3,726,MultiVehicle Crash,"DAVIDSON on Interstate 40 Westbound, Beyond MI...",2020-11-12,10:25:06 AM,100,0,0,0,...,1017.7,5.1,9.9,2020-11-12T06:21:29,2020-11-12T16:40:48,0.95,Clear,Clear conditions throughout the day.,Clear,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
1,3,728,Disabled Vehicle,"DAVIDSON on Interstate 65 Northbound, Before M...",2020-11-12,10:35:05 AM,102.3666667,0,0,0.066666667,...,1017.7,5.1,9.9,2020-11-12T06:21:29,2020-11-12T16:40:48,0.95,Clear,Clear conditions throughout the day.,Clear,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"


In [19]:
multivehicle_rain_df = traffic_df.loc[(traffic_df['Incident Type'] == 'MultiVehicle Crash') & (traffic_df['conditions'] == 'Rain')]

In [20]:
multivehicle_rain_df

,Region,Incident ID,Incident Type,Full Location,Start Date,Start Time,Duration ICT,Lanes Blocked,Blockage Duration RCT,Response Time,...,sealevelpressure,cloudcover,visibility,sunrise,sunset,moonphase,conditions,description,icon,stations
240,3,1002,MultiVehicle Crash,"WILLIAMSON on Interstate 65 Southbound, Beyond...",2020-11-15,2:56:20 AM,29.26666667,0,0,0,...,1016.0,43.6,9.7,2020-11-15T06:24:32,2020-11-15T16:38:44,0.00,Rain,Clearing in the afternoon with rain clearing l...,Rain,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
244,3,1010,MultiVehicle Crash,"DAVIDSON on Interstate 40 Westbound, Beyond MI...",2020-11-15,6:46:25 AM,54,1,10,2,...,1016.0,43.6,9.7,2020-11-15T06:24:32,2020-11-15T16:38:44,0.00,Rain,Clearing in the afternoon with rain clearing l...,Rain,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
246,3,1012,MultiVehicle Crash,"DAVIDSON on Interstate 40 Westbound, Before MI...",2020-11-15,6:54:25 AM,54,2,52,-17,...,1016.0,43.6,9.7,2020-11-15T06:24:32,2020-11-15T16:38:44,0.00,Rain,Clearing in the afternoon with rain clearing l...,Rain,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
247,3,1013,MultiVehicle Crash,"DAVIDSON on Interstate 24 Eastbound, Beyond MI...",2020-11-15,6:58:18 AM,16,1,14,2,...,1016.0,43.6,9.7,2020-11-15T06:24:32,2020-11-15T16:38:44,0.00,Rain,Clearing in the afternoon with rain clearing l...,Rain,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
248,3,1014,MultiVehicle Crash,"DAVIDSON on Interstate 24 Eastbound, Before MI...",2020-11-15,7:02:54 AM,87,2,52,-18,...,1016.0,43.6,9.7,2020-11-15T06:24:32,2020-11-15T16:38:44,0.00,Rain,Clearing in the afternoon with rain clearing l...,Rain,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33643,3,106512,MultiVehicle Crash,"GILES on Interstate 65 Southbound, Beyond MILE...",2021-10-30,8:41:07 PM,481.5666667,1,480.55,0,...,1004.7,97.5,7.6,2021-10-30T07:08:14,2021-10-30T17:52:46,0.84,Rain,Cloudy skies throughout the day with rain in t...,Rain,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
33644,3,106513,MultiVehicle Crash,"DAVIDSON on Interstate 65 Southbound, At EXIT ...",2021-10-30,8:53:17 PM,30.4,0,28.066667,1.65,...,1004.7,97.5,7.6,2021-10-30T07:08:14,2021-10-30T17:52:46,0.84,Rain,Cloudy skies throughout the day with rain in t...,Rain,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
33645,3,106514,MultiVehicle Crash,"DAVIDSON on Interstate 24 Westbound, Beyond MI...",2021-10-30,9:44:55 PM,72,0,0.0,0,...,1004.7,97.5,7.6,2021-10-30T07:08:14,2021-10-30T17:52:46,0.84,Rain,Cloudy skies throughout the day with rain in t...,Rain,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
33646,3,106515,MultiVehicle Crash,"DAVIDSON on Interstate 65 Southbound, Before M...",2021-10-30,10:03:31 PM,68.36666667,2,15.766667,3.65,...,1004.7,97.5,7.6,2021-10-30T07:08:14,2021-10-30T17:52:46,0.84,Rain,Cloudy skies throughout the day with rain in t...,Rain,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"


### Reviewing Traffic Columns

In [21]:
traffic_df.columns

Index(['Region', 'Incident ID', 'Incident Type', 'Full Location ',
       'Start Date', 'Start Time', 'Duration ICT', 'Lanes Blocked',
       'Blockage Duration RCT', 'Response Time', 'Rural/\nUrban', 'Key ',
       'County', 'Interstate', 'Route', 'Direction', 'Unnamed: 16',
       'Description', 'Inc MM', 'Latitude', 'Longitude', 'Severity Lanes',
       'Severity Duration', 'Severity', 'Temp', 'Prec', 'Condition', 'name',
       'datetime', 'tempmax', 'tempmin', 'temp', 'feelslikemax',
       'feelslikemin', 'feelslike', 'dew', 'humidity', 'precip', 'precipcover',
       'snow', 'snowdepth', 'windgust', 'windspeed', 'winddir',
       'sealevelpressure', 'cloudcover', 'visibility', 'sunrise', 'sunset',
       'moonphase', 'conditions', 'description', 'icon', 'stations'],
      dtype='object')

### Calculating the frequencies in the dataset

In [22]:
incidents = traffic_df['Incident Type'].value_counts().reset_index()
incidents.columns = ['Incident Type', 'Count']

In [23]:
#calculate the max amount of incidents
incidents_max = incidents['Incident Type'].iloc[incidents['Count'].idxmax()]

In [24]:
#printing out the max amount of incidents
print('The ' + str(incidents_max) + ' has the most incidents in the dataset.')

The Disabled Vehicle has the most incidents in the dataset.


In [25]:
#view the dataframe
incidents.head()

,Incident Type,Count
0,Disabled Vehicle,19210
1,MultiVehicle Crash,5493
2,Abandoned Vehicle,4841
3,Debris on Roadway,1908
4,Single Vehicle Crash,1216


In [26]:
#spot check
traffic_df['Incident Type'].value_counts()
#traffic_df['Incident Type'].isnull().value_counts()
#traffic_df['Incident Type'].unique()

Disabled Vehicle              19210
MultiVehicle Crash             5493
Abandoned Vehicle              4841
Debris on Roadway              1908
Single Vehicle Crash           1216
PD/Med/Fire Activity            376
Overturned Vehicle              271
Vehicle Fire                    190
Weather                          54
Flooding                         36
Special Event                    35
Jack-Knife Tractor Trailer       33
Other                            28
Off Ramp Backup                  22
Interstate Shooting              13
Oversize Load                     2
Wrong Way Driver                  2
Name: Incident Type, dtype: int64

In [27]:
countys = traffic_df['County'].value_counts().reset_index()
countys.columns = ['County', 'Count']

In [28]:
countys.head()

,County,Count
0,DAVIDSON,28129
1,RUTHERFORD,3150
2,WILLIAMSON,1650
3,WILSON,276
4,CHEATHAM,159


In [29]:
davidson_shooting_df = traffic_df.loc[(traffic_df['County']=='DAVIDSON') & (traffic_df['Incident Type'] == 'Interstate Shooting')]

In [30]:
davidson_shooting_df

,Region,Incident ID,Incident Type,Full Location,Start Date,Start Time,Duration ICT,Lanes Blocked,Blockage Duration RCT,Response Time,...,sealevelpressure,cloudcover,visibility,sunrise,sunset,moonphase,conditions,description,icon,stations
429,3,1254,Interstate Shooting,"DAVIDSON on Interstate 65 Northbound, Before M...",2020-11-17,12:35:50 PM,43.05,0,0,0.033333333,...,1031.1,2.2,9.9,2020-11-17T06:26:34,2020-11-17T16:37:30,0.04,Clear,Clear conditions throughout the day.,Clear,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
973,3,2035,Interstate Shooting,"DAVIDSON on Interstate 24 Eastbound, Before MI...",2020-11-24,3:15:35 AM,297,4,84,0,...,1022.4,50.5,9.4,2020-11-24T06:33:34,2020-11-24T16:34:11,0.36,Clear,Partly cloudy throughout the day.,Clear,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
1738,3,3009,Interstate Shooting,"DAVIDSON on Interstate 440 Westbound, Before M...",2020-12-03,8:59:31 PM,319,3,52,0,...,1024.7,77.8,9.5,2020-12-03T06:42:00,2020-12-03T16:32:15,0.55,Rain,Partly cloudy throughout the day with late aft...,Rain,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
3066,3,4811,Interstate Shooting,"DAVIDSON on Interstate 40 Westbound, Beyond MI...",2020-12-19,5:50:39 PM,10,0,0,0,...,1023.6,81.5,9.3,2020-12-19T06:53:42,2020-12-19T16:35:26,0.13,Rain,Partly cloudy throughout the day with late aft...,Rain,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
11782,3,15962,Interstate Shooting,"DAVIDSON on Interstate 65 Southbound, Before M...",2021-03-27,12:49:36 AM,80.63333333,1,36.73333333,0,...,1015.6,79.9,7.7,2021-03-27T06:40:02,2021-03-27T19:05:13,0.49,Rain,Partly cloudy throughout the day with a chance...,Rain,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
12774,3,17261,Interstate Shooting,"DAVIDSON on Interstate 65 Southbound, Before M...",2021-04-05,8:33:31 PM,53.55,1,22.31666667,0.05,...,1019.5,13.9,9.9,2021-04-05T06:27:07,2021-04-05T19:12:50,0.81,Rain,Clear conditions throughout the day with rain ...,Rain,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
15948,3,21342,Interstate Shooting,"DAVIDSON on Interstate 24 Eastbound, Before MI...",2021-05-08,9:27:10 PM,3,0,0,0,...,1017.8,73.8,9.9,2021-05-08T05:46:48,2021-05-08T19:40:56,0.96,Clear,Partly cloudy throughout the day.,Clear,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
16024,3,21441,Interstate Shooting,"DAVIDSON on Interstate 24 Westbound, Beyond MI...",2021-05-10,8:29:56 AM,240,1,18,0,...,1018.3,89.6,9.7,2021-05-10T05:44:58,2021-05-10T19:42:36,1.00,Rain,Partly cloudy throughout the day with early mo...,Rain,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
16027,3,21446,Interstate Shooting,"DAVIDSON on Interstate 24 Westbound, Before MI...",2021-05-10,9:02:15 AM,196,2,49,1,...,1018.3,89.6,9.7,2021-05-10T05:44:58,2021-05-10T19:42:36,1.00,Rain,Partly cloudy throughout the day with early mo...,Rain,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
27375,3,36561,Interstate Shooting,"DAVIDSON on Interstate 65 Northbound, At EXIT ...",2021-09-02,6:52:02 PM,42.88333333,0,39.6,0.016666667,...,1015.4,12.2,9.9,2021-09-02T06:19:41,2021-09-02T19:13:05,0.90,Clear,Clear conditions throughout the day.,Clear,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"


In [31]:
tractor_trailer_df = traffic_df.loc[traffic_df['Incident Type']=='Jack-Knife Tractor Trailer']

In [32]:
tractor_trailer_df

,Region,Incident ID,Incident Type,Full Location,Start Date,Start Time,Duration ICT,Lanes Blocked,Blockage Duration RCT,Response Time,...,sealevelpressure,cloudcover,visibility,sunrise,sunset,moonphase,conditions,description,icon,stations
3976,3,5924,Jack-Knife Tractor Trailer,"DAVIDSON on Interstate 40 Eastbound, Before MI...",2020-12-30,11:08:05 PM,52,2,52,12,...,1019.4,70.3,9.1,2020-12-30T06:57:59,2020-12-30T16:42:05,0.50,Rain,Partly cloudy throughout the day with late aft...,Rain,"72327013897,KJWN,KBNA,KM91,KMQY,72327313827"
4655,3,6770,Jack-Knife Tractor Trailer,"WILLIAMSON on Interstate 65 Northbound, Before...",2021-01-07,10:22:33 PM,139.85,2,139.35,-0.15,...,1019.5,80.8,9.9,2021-01-07T06:58:44,2021-01-07T16:48:41,0.82,Clear,Partly cloudy throughout the day.,Clear,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
4905,3,7081,Jack-Knife Tractor Trailer,"RUTHERFORD on Interstate 24 Westbound, Beyond ...",2021-01-11,5:01:19 PM,148,0,0,0,...,1027.6,94.7,7.5,2021-01-11T06:58:19,2021-01-11T16:52:24,0.99,Snow,Cloudy skies throughout the day with late afte...,Snow,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
7570,3,10478,Jack-Knife Tractor Trailer,"WILSON on Interstate 40 Westbound, Beyond MILE...",2021-02-14,6:38:43 AM,126,2,65,5,...,1023.7,99.5,2.6,2021-02-14T06:35:21,2021-02-14T17:27:41,0.04,Snow,Cloudy skies throughout the day with late afte...,Snow,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
7596,3,10510,Jack-Knife Tractor Trailer,"RUTHERFORD on Interstate 24 Westbound, Before ...",2021-02-14,9:58:56 AM,146,2,21,0,...,1023.7,99.5,2.6,2021-02-14T06:35:21,2021-02-14T17:27:41,0.04,Snow,Cloudy skies throughout the day with late afte...,Snow,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
7715,3,10650,Jack-Knife Tractor Trailer,"WILSON on Interstate 40 Westbound, Beyond MILE...",2021-02-15,10:13:42 AM,260,2,67,0,...,1015.7,95.7,3.5,2021-02-15T06:34:15,2021-02-15T17:28:42,0.07,Snow,Cloudy skies throughout the day with a chance ...,Snow,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
8039,3,11039,Jack-Knife Tractor Trailer,"WILSON on Interstate 40 Westbound, Beyond MILE...",2021-02-17,7:38:24 PM,49,2,48,0,...,1025.3,88.1,7.3,2021-02-17T06:31:58,2021-02-17T17:30:43,0.14,Snow,Partly cloudy throughout the day with rain or ...,Snow,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
8098,3,11111,Jack-Knife Tractor Trailer,"WILLIAMSON on Interstate 65 Southbound, Beyond...",2021-02-18,6:33:58 AM,208.1833333,2,203.1666667,37.06666667,...,1023.1,96.5,5.7,2021-02-18T06:30:48,2021-02-18T17:31:43,0.19,Snow,Cloudy skies throughout the day with a chance ...,Snow,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
8472,3,11564,Jack-Knife Tractor Trailer,"RUTHERFORD on Interstate 24 Westbound, Before ...",2021-02-22,10:48:22 AM,196,1,59,-6,...,1017.7,54.5,9.5,2021-02-22T06:25:58,2021-02-22T17:35:39,0.38,Snow,Clearing in the afternoon with early morning s...,Snow,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
12221,3,16569,Jack-Knife Tractor Trailer,"DAVIDSON on Interstate 65 Southbound, Beyond M...",2021-03-31,6:40:13 AM,416,4,414.7666667,5.95,...,1020.8,78.2,8.9,2021-03-31T06:34:15,2021-03-31T19:08:36,0.55,Rain,Partly cloudy throughout the day with a chance...,Rain,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"


In [33]:
vehicle_fire_df = traffic_df.loc[traffic_df['Incident Type']=='Vehicle Fire']

In [34]:
vehicle_fire_df

,Region,Incident ID,Incident Type,Full Location,Start Date,Start Time,Duration ICT,Lanes Blocked,Blockage Duration RCT,Response Time,...,sealevelpressure,cloudcover,visibility,sunrise,sunset,moonphase,conditions,description,icon,stations
239,3,1001,Vehicle Fire,"RUTHERFORD on Interstate 24 Eastbound, Beyond ...",2020-11-14,10:12:44 PM,157,3,152,2,...,1018.2,68.9,9.9,2020-11-14T06:23:31,2020-11-14T16:39:24,1.00,Clear,Partly cloudy throughout the day.,Clear,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
439,3,1276,Vehicle Fire,"DAVIDSON on Interstate 65 Northbound, Before M...",2020-11-17,4:06:42 PM,581.05,3,47.31666667,0.016666667,...,1031.1,2.2,9.9,2020-11-17T06:26:34,2020-11-17T16:37:30,0.04,Clear,Clear conditions throughout the day.,Clear,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
518,3,1401,Vehicle Fire,"CHEATHAM on Interstate 24 Westbound, Before MI...",2020-11-18,3:03:54 PM,1,0,0,0,...,1034.6,4.8,9.9,2020-11-18T06:27:35,2020-11-18T16:36:56,0.07,Clear,Clear conditions throughout the day.,Clear,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
662,3,1590,Vehicle Fire,"CHEATHAM on Interstate 24 Eastbound, Beyond MI...",2020-11-20,5:00:24 AM,164,1,162,-164,...,1030.5,32.1,9.8,2020-11-20T06:29:35,2020-11-20T16:35:54,0.17,Clear,Partly cloudy throughout the day.,Clear,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
1063,3,2152,Vehicle Fire,"DAVIDSON on Interstate 65 Southbound, Beyond M...",2020-11-24,9:48:39 PM,49.96666667,2,49.2,0.516666667,...,1022.4,50.5,9.4,2020-11-24T06:33:34,2020-11-24T16:34:11,0.36,Clear,Partly cloudy throughout the day.,Clear,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33268,3,105988,Vehicle Fire,"DAVIDSON on Interstate 65 Southbound, Before M...",2021-10-26,7:39:18 PM,11.21666667,0,1.516667,2.116666667,...,1016.9,64.8,9.9,2021-10-26T07:04:23,2021-10-26T17:57:10,0.65,Clear,Partly cloudy throughout the day.,Clear,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
33310,3,106063,Vehicle Fire,"WILLIAMSON on Interstate 840 Eastbound, At MIL...",2021-10-27,2:16:30 PM,26,1,25.0,0,...,1012.4,38.6,9.9,2021-10-27T07:05:21,2021-10-27T17:56:02,0.69,Clear,Partly cloudy throughout the day.,Clear,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
33331,3,106088,Vehicle Fire,"RUTHERFORD on Interstate 24 Eastbound, Beyond ...",2021-10-27,6:04:14 PM,38,0,1.0,0,...,1012.4,38.6,9.9,2021-10-27T07:05:21,2021-10-27T17:56:02,0.69,Clear,Partly cloudy throughout the day.,Clear,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
33332,3,106089,Vehicle Fire,"RUTHERFORD on Interstate 24 Eastbound, At MILE...",2021-10-27,6:15:12 PM,4,0,0.0,0,...,1012.4,38.6,9.9,2021-10-27T07:05:21,2021-10-27T17:56:02,0.69,Clear,Partly cloudy throughout the day.,Clear,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"


In [35]:
wrong_way_df = traffic_df.loc[traffic_df['Incident Type']=='Wrong Way Driver']

In [36]:
wrong_way_df

,Region,Incident ID,Incident Type,Full Location,Start Date,Start Time,Duration ICT,Lanes Blocked,Blockage Duration RCT,Response Time,...,sealevelpressure,cloudcover,visibility,sunrise,sunset,moonphase,conditions,description,icon,stations
14054,3,18907,Wrong Way Driver,"DAVIDSON on Interstate 65 Southbound, Before M...",2021-04-19,2:01:45 AM,88.28333333,0,0,0,...,1014.9,31.2,9.7,2021-04-19T06:08:11,2021-04-19T19:24:43,0.21,Clear,Partly cloudy throughout the day.,Clear,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
25541,3,34071,Wrong Way Driver,"DAVIDSON on Interstate 40 Eastbound, Before MI...",2021-08-15,11:12:18 PM,67,1,8,0,...,1017.9,74.0,9.3,2021-08-15T06:05:31,2021-08-15T19:37:00,0.24,Rain,Partly cloudy throughout the day with rain.,Rain,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"


In [37]:
conditions = traffic_df['conditions'].value_counts().reset_index()
conditions.columns = ['Conditions', 'Count']

In [38]:
conditions_max = conditions['Conditions'].iloc[conditions['Count'].idxmax()]

In [39]:
conditions.head()

,Conditions,Count
0,Clear,20272
1,Rain,12754
2,Snow,704


In [40]:
conditions_max

'Clear'

In [41]:
#printing out the max amount of conditions
print('The ' + str(conditions_max) + ' condition has the greatest amount of incidents in the dataset.')

The Clear condition has the greatest amount of incidents in the dataset.


In [42]:
peak_time_df = traffic_df['Start Time'].value_counts().reset_index()
peak_time_df.columns = ['Start Time', 'Count']

In [43]:
peak_time_df.head(20)

,Start Time,Count
0,2:52:53 PM,6
1,5:58:14 PM,5
2,5:29:35 PM,5
3,3:16:14 PM,5
4,6:45:54 AM,5
5,6:10:22 PM,5
6,3:39:56 PM,5
7,2:35:36 PM,5
8,12:45:15 PM,5
9,2:50:37 PM,5


In [44]:
days_df = traffic_df['Start Date'].value_counts().reset_index()
days_df.columns = ['Start Date', 'Count']

In [45]:
days_df.head(40)

,Start Date,Count
0,2021-07-09,146
1,2021-04-07,145
2,2021-03-02,141
3,2021-07-29,140
4,2021-02-22,140
5,2021-08-12,138
6,2021-08-06,137
7,2021-02-15,137
8,2021-08-26,136
9,2021-09-21,134


In [49]:
crashes_df = traffic_df.loc[(traffic_df['Incident Type']=='Single Vehicle Crash') & (traffic_df['Incident Type'] == 'MultiVehicle Crash')]

Ask how to group the Start Times into hourly buckets to have value count frequency. 
rolling buckets would produce the most accurate. 
Ask how to group the peak period with Start Time & Start Date values

In [54]:
traffic_df.head(2)

,Region,Incident ID,Incident Type,Full Location,Start Date,Start Time,Duration ICT,Lanes Blocked,Blockage Duration RCT,Response Time,...,sealevelpressure,cloudcover,visibility,sunrise,sunset,moonphase,conditions,description,icon,stations
0,3,726,MultiVehicle Crash,"DAVIDSON on Interstate 40 Westbound, Beyond MI...",2020-11-12,10:25:06 AM,100,0,0,0,...,1017.7,5.1,9.9,2020-11-12T06:21:29,2020-11-12T16:40:48,0.95,Clear,Clear conditions throughout the day.,Clear,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"
1,3,728,Disabled Vehicle,"DAVIDSON on Interstate 65 Northbound, Before M...",2020-11-12,10:35:05 AM,102.3666667,0,0,0.066666667,...,1017.7,5.1,9.9,2020-11-12T06:21:29,2020-11-12T16:40:48,0.95,Clear,Clear conditions throughout the day.,Clear,"72327013897,KJWN,KBNA,KMQY,KM91,72327313827"


In [62]:
traffic_df['Start Datetime']=pd.to_datetime(traffic_df['Start Date'].astype(str) + " " + traffic_df['Start Time'])

In [68]:
peak_period_df = traffic_df.groupby(pd.Grouper(key='Start Datetime', freq='d'))['Incident ID'].count()

In [69]:
peak_period_df

Start Datetime
2020-11-01     60
2020-11-02     87
2020-11-03    104
2020-11-04     97
2020-11-05    101
             ... 
2021-10-27     75
2021-10-28    111
2021-10-29    105
2021-10-30     83
2021-10-31     82
Freq: D, Name: Incident ID, Length: 365, dtype: int64

In [ ]:
#keep code below for future plotting
#traffic_df.groupby(pd.Grouper(key='Start Datetime', freq='d'))['Incident ID'].count().plot(kind='line')
#traffic_df.groupby(pd.Grouper(key='Start Datetime', freq='h'))['Incident ID'].count().sort_values(ascending = False)

### Top _ sorted & Column rename

In [ ]:
#gdp_and_internet_use_2014.sort_values(by = 'Internet_Users_Pct', ascending = False).head()

In [ ]:
#d_weather = d_weather.rename(columns = {'Start Date':'Start_Date'})

In [ ]:
#traffic_df['Start Date'] = traffic_df['Start Date'].astype('str')

### Bar Chart

In [ ]:
#plotting the greatest frequency of incidents
plt.figure(figsize = (14,10))
sns.barplot(x = 'Incident Type', y = 'Count', data = incidents)
plt.xticks(rotation=65, horizontalalignment='center')
plt.xlabel('Incident Type', fontsize = 15)
plt.ylabel('Total Incidents', fontsize = 15)
plt.title('Traffic Incidents Based on their Frequency', fontsize = 20);

In [ ]:
#plotting the greatest frequency of incidents per county
plt.figure(figsize = (14,10))
sns.barplot(x = 'County', y = 'Count', data = countys)
plt.xticks(rotation=65, horizontalalignment='center')
plt.xlabel('County', fontsize = 15)
plt.ylabel('Total Incidents', fontsize = 15)
plt.title('Total Frequency of Incidents Based on County', fontsize = 20);

### Histographs

In [ ]:
#traffic_df['column name'].value_counts()

In [ ]:
#traffic_df['number_of_motor_vehicles'] = pd.to_numeric(traffic_df['number_of_motor_vehicles'])  
                                                                        
#traffic_df.hist(['number_of_motor_vehicles'])

In [ ]:
#car_accidents_2010_2020['DateTime'] = pd.to_datetime(car_accidents_2010_2020['Date and Time'])
#car_accidents_2010_2020['date'] = car_accidents_2010_2020['DateTime'].dt.date
#car_accidents_2010_2020['time'] = car_accidents_2010_2020['DateTime'].dt.time

In [ ]:
#car_accidents_2010_2020 = car_accidents_2010_2020.set_index('DateTime')
#car_accidents_2010_2020.head(3)

In [ ]:
#car_accidents_2010_2020['Collision Type'].value_counts(normalize=True).sort_index().plot.bar()
#plt.grid()
#plt.title('collision Type')
#plt.xlabel('Collision Type')
#plt.ylabel('Fraction');

In [ ]:
#car_accidents_2010_2020['Collision Type'].value_counts().plot.bar()
#plt.grid()
#plt.title('collision Type')
#plt.xlabel('Collision Type')
#plt.ylabel('Fraction');

In [ ]:
#car_accidents_2010_2020['Collision Type Description'].value_counts(normalize=True).sort_index().plot.bar()
#plt.grid()
#plt.title('Collision Type Description')
#plt.xlabel('Collision Type Description')
#plt.ylabel('Fraction');

In [ ]:
#car_accidents_2010_2020['Collision Type Description'].value_counts().plot.bar()
#plt.grid()
#plt.title('Collision Type Description')
#plt.xlabel('Collision Type Description')
#plt.ylabel('Total count');

In [ ]:
#car_accidents_2010_2020['Weekday'] = car_accidents_2010_2020.index.weekday_name
#weekday = car_accidents_2010_2020.groupby('Weekday')['Accident Number'].count()
#weekday = weekday/weekday.sum()
#dayOfWeek=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
#weekday[dayOfWeek].plot.bar()
#plt.title('Acccidents by Weekday')
#plt.xlabel('Weekday')
#plt.ylabel('Accidents');

In [ ]:
#years = ['2010','2011','2012','2013', '2014', '2015', '2016', '2017', '2018', '2019']
#fig, _ = plt.subplots(1,10,figsize=(21,7), sharex='all', sharey='all')

#fig.suptitle('Acccidents by Weekday for Different Years')
#plt.xlabel('Weekday')
#plt.ylabel('Accidents')
#for i, year in enumerate(years,1):
   # weekday = car_accidents_2010_2020.loc[year].groupby('Weekday')['Accident Number'].count()
  #  weekday = weekday/weekday.sum()
  #  dayOfWeek=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
  #  plt.subplot(1,10,i)
  #  plt.title(year)
  #  weekday[dayOfWeek].plot.bar()

### Maps and Scatterplots

In [ ]:
plt.figure(figsize = (15,10))
plt.scatter(x=traffic_df['Longitude'], y=traffic_df['Latitude'])
plt.title('Traffic Incidents in Nashville, TN', fontsize = 15)
plt.xlabel('Longitude Coordinates', fontsize = 12)
plt.ylabel('Latitude Coordinates', fontsize = 12)
plt.show()

In [ ]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame

#traffic_df = pd.read_csv("Long_Lats.csv", delimiter=',', skiprows=0, low_memory=False)

geometry = [Point(xy) for xy in zip(traffic_df['Longitude'], traffic_df['Latitude'])]
gdf = GeoDataFrame(traffic_df, geometry=geometry)   

#this is a simple map that goes with geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
ax = world[world.continent == 'North America'].plot(
    color='white', edgecolor='black')
#ax = gdf.plot(figsize = (10, 6), color = 'red')
gdf.plot(ax=ax, color='red', marker='o')
#code for the world below
#gdf.plot(ax=world.plot(figsize=(10, 6)), marker='o', color='red', markersize=15);

In [ ]:
#center = traffic_df.geometry.centroid[-86.77676762, 36.2324525740]

In [ ]:
#center.y = 36.2324525740

In [ ]:
#center.x = -86.77676762

In [ ]:
area_center = [36.2324525740, -86.77676762]

In [ ]:
print(area_center)

In [ ]:
#only run if needed
#nash_map = folium.Map(location = area_center, zoom_start = 12)

#for row_index, row_values in traffic_df.iterrows():
#    loc = [row_values['Latitude'], row_values['Longitude']]
 #   pop = str(row_values['County'])
#    icon = folium.Icon(color = "red", icon ="traffic", prefix='fa')
    
   # marker = folium.Marker(
  #      location = loc, 
  #      popup = pop, icon = icon) 

 #   marker.add_to(nash_map)
#nash_map.save('../data/nash_map.html')


In [ ]:
#nash_map

In [ ]:
cluster_map_nash = folium.Map(location =  area_center, zoom_start = 12)

#create a marker cluster
marker_cluster = MarkerCluster().add_to(cluster_map_nash)

#folium.GeoJson(traffic_df).add_to(cluster_map_nash)

# inside the loop add each marker to the cluster
for row_index, row_values in traffic_df.iterrows():
    loc = [row_values['Latitude'], row_values['Longitude']]
    pop = str(row_values['County'])
    icon=folium.Icon(color = "blue", icon ="traffic", prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop,
    icon = icon) 
    
    marker.add_to(marker_cluster)

    
#save an interactive HTML map by calling .save()
#cluster_map_nash.save('../data/cluster_map_nash.html')

In [ ]:
cluster_map_nash

### Creating a cluster map of the incident locations in Nashville

In [ ]:
cluster_map_nash = folium.Map(location =  area_center, zoom_start = 12)

#create a list of locations and pass them to FastMarkerCluster()
locations = traffic_df[['Latitude', 'Longitude']].values.tolist()
cluster_map_nash.add_child(
    FastMarkerCluster(locations)
)

In [ ]:
#save an interactive HTML map by calling .save()
cluster_map_nash.save('../data/cluster_map_nash.html')

In [ ]:
#cluster
cluster_map_nash2 = folium.Map(location =  area_center, zoom_start = 12)

#create a list of locations and pass them to FastMarkerCluster()
conditions_nash = traffic_df[['Latitude', 'Longitude']].values.tolist()
cluster_map_nash2.add_child(
    FastMarkerCluster(conditions_nash)
)

In [ ]:
#cluster of mutlivehicle crashes
cluster_map_nash3 = folium.Map(location =  area_center, zoom_start = 12)

#create a list of locations and pass them to FastMarkerCluster()
multivehicle_rain_nash = multivehicle_rain_df[['Latitude', 'Longitude']].values.tolist()
cluster_map_nash3.add_child(
    FastMarkerCluster(multivehicle_rain_nash)
)

In [ ]:
#cluster tractor trailer
cluster_map_nash4 = folium.Map(location =  area_center, zoom_start = 12)

#create a list of locations and pass them to FastMarkerCluster()
tractor_trailer_df = tractor_trailer_df[['Latitude', 'Longitude']].values.tolist()
cluster_map_nash4.add_child(
    FastMarkerCluster(tractor_trailer_df)
)

In [ ]:
#cluster shooting
cluster_map_nash5 = folium.Map(location =  area_center, zoom_start = 12)

#create a list of locations and pass them to FastMarkerCluster()
davidson_shooting_df = davidson_shooting_df[['Latitude', 'Longitude']].values.tolist()
cluster_map_nash5.add_child(
    FastMarkerCluster(davidson_shooting_df)
)

In [ ]:
#plt.figure(figsize = (15,10))
#sns.scatterplot(x = 'Internet_Users_Pct', y = 'GDP_Per_Capita', data = gdp_and_internet_use_2004, alpha = 0.9)
#plt.title('2004 Percent Using the Internet vs GDP Per Capita', fontsize = 15)
#plt.xlabel('Percentage of Internet Users', fontsize = 12)
#plt.ylabel('GDP Per Capita', fontsize = 12)

In [ ]:
# distribution of gdp per capita 
#ax = sns.distplot(gdp_and_internet_use.gdp_percapita,
#                 bins = 25,
#                  hist_kw={'alpha' :.2})
#ax.set(xlabel = 'GDP per Capita', ylabel = 'Frequency')

In [ ]:
#internet use
#ax = sns.distplot(gdp_and_internet_use.Internet_Users_Pct,
#                 bins = 25,
#                  color  = 'green',
#                  hist_kw={'alpha':.4})
#ax.set(xlabel = 'Internet User Pct', ylabel = 'Frequency')

In [ ]:
#years = ['2010','2011','2012','2013', '2014', '2015', '2016', '2017', '2018', '2019']
#fig, _ = plt.subplots(1,10, figsize=(44,8), sharex='all', sharey='all')

#fig.suptitle('Acccidents by month for Different Years')
#plt.xlabel('month')
#plt.ylabel('Accidents')
#for i, year in enumerate(years,1):
 #   plt.subplot(1,10,i)
 #   sample = car_accidents_2010_2020.loc[year]['Accident Number'].resample('M').count()
 #   sample.plot()
 #   plt.ylim(0,8000)
 #   plt.title('Accidents, {} count'.format(text))
#    plt.xlabel('Year')
#    plt.ylabel('Accident Count');

#### Code examples to keep

In [ ]:
#geospatial coding. update 'geometry' column
#traffic_df['geometry'] = traffic_df.apply(lambda x: Point((float(x.Longitude),
 #                                                  float(x.Latitude))),
  #                                axis=1)

In [ ]:
#type(traffic_geo)
#traffic_geo = gpd.GeoDataFrame(traffic_df)
#traffic_geo.head()
#type(traffic_geo)

In [ ]:
#rows_and_cols = traffic_df.shape
#print('There are {} rows and {} columns.\n'.format(
  #  rows_and_cols[0], rows_and_cols[1]))

In [ ]:
# compile the list of dataframes to be merged
#data_frames = [car_accident0, car_accident1, car_accident2,car_accident3, car_accident4]

#traffic_2021 = pd.concat([traffic_df, h_weather, d_weather], sort=True)
#traffic_2021.head(2)

In [ ]:
#The amount in brackets will populate. 
#print('There are {} accidents in the data'.format(len(car_accidents_2010_2020)))

In [ ]:
#dropping columns, all columns listed to have as a visual
#pd.set_option('display.max_columns', 1000)
#df.drop(['Region', 'Incident ID', 'Incident Type', 'Full Location ',
#       'Start Date', 'Start Time', 'Duration ICT', 'Lanes Blocked RCT',
#       'Blockage\nDuration', 'Resp Time', 'Rural/\nUrban', 'Key ', 'County',
#       'Interstate', 'Route', 'Direction', 'Unnamed: 16', 'Description',
#       'Inc MM', 'Latitude', 'Longitude', 'Severity Lanes',
#       'Severity Duration', 'Severity'], axis=1, inplace=True)
#df.tail()

In [ ]:
#Extracting just the digits and the decimal place
#gourds['weight'] = gourds['weight'].str.extract('(\d*.\d*)')

#Making sure it's a float
#gourds['weight'] = gourds['weight'].astype('float64')

#Sorting based on weight column
#gourds = gourds.sort_values(by = 'weight', ascending = False).reset_index(drop = True)

## Extracting the dataframe to import into dashboard

In [ ]:
#extracting dataframe into a csv file
traffic_df.to_csv('../data/MASTER File 2021 Tableau.csv')

In [ ]:
rain_df.to_csv('../data/rain.csv')

In [ ]:
snow_df.to_csv('../data/snow.csv')

In [ ]:
clear_df.to_csv('../data/clear.csv')

In [ ]:
incidents.to_csv('../data/incidents.csv')

In [ ]:
conditions.to_csv('../data/conditions.csv')

In [ ]:
multivehicle_rain_df.to_csv('../data/multivehicle_rain.csv')

In [ ]:
davidson_shooting_df.to_csv('../data/davidson_shooting.csv')

In [ ]:
tractor_trailer_df.to_csv('../data/tractor_trailer.csv')

In [ ]:
vehicle_fire_df.to_csv('../data/vehicle_fire.csv')

In [ ]:
wrong_way_df.to_csv('../data/wrong_way.csv')

In [ ]:
peak_time_df.to_csv('../data/peak_time.csv')

In [ ]:
days_df.to_csv('../data/days.csv')

In [70]:
peak_period_df.to_csv('../data/peak_period.csv')

In [71]:
crashes_df.to_csv('../data/crashes.csv')